# Details
This notebook was used for a short mesh compression demo that I developed as part of CSC 461.

You can find more details at https://anormananderson.github.io/real-time-compression/ as well as https://www.youtube.com/watch?v=8Caw7d_83wE

# Initial terminology

Vertex: A point in space.

Edge: An edge connects two vertices. It is a line between two points.

Face: For our purposes, an individual triangle.

Mesh: A collection of vertices, edges, and faces which together describe some polyhedral object.

Parameterization: In our case, we are mapping vertices into the 2D u, v coordinate plane.
This means that we define a function f(u, v) which spits out the 3D coordinate represented
by the point at (u, v) in the 2D plane.

Genus: The number of holes in a surface.

# Goals

Our goals for this project are to:

- Load a 3D object from a file.

- Parameterize it into the 2D plane.

- Quantize every vertex position on the 2D plane

In [1]:
import igl
import meshplot as mp
import numpy as np
from PIL import Image
import pythreejs as p3s

In [2]:
# This mesh is available in the data folder for the libigl Python tutorials.
# That data is available at https://github.com/libigl/libigl-tutorial-data
v, f = igl.read_triangle_mesh("data/camelhead.off")

In [3]:
''' (A libigl function: https://github.com/libigl/libigl/blob/main/include/igl/map_vertices_to_circle.cpp)
// This file is part of libigl, a simple c++ geometry processing library.
//
// Copyright (C) 2014 Stefan Brugger <stefanbrugger@gmail.com>
//
// This Source Code Form is subject to the terms of the Mozilla Public License
// v. 2.0. If a copy of the MPL was not distributed with this file, You can
// obtain one at http://mozilla.org/MPL/2.0/.

#include "map_vertices_to_circle.h"
#include "PI.h"

IGL_INLINE void igl::map_vertices_to_circle(
  const Eigen::MatrixXd& V,
  const Eigen::VectorXi& bnd,
  Eigen::MatrixXd& UV)
{
  // Get sorted list of boundary vertices
  std::vector<int> interior,map_ij;
  map_ij.resize(V.rows());

  std::vector<bool> isOnBnd(V.rows(),false);
  for (int i = 0; i < bnd.size(); i++)
  {
    isOnBnd[bnd[i]] = true;
    map_ij[bnd[i]] = i;
  }

  for (int i = 0; i < (int)isOnBnd.size(); i++)
  {
    if (!isOnBnd[i])
    {
      map_ij[i] = interior.size();
      interior.push_back(i);
    }
  }

  // Map boundary to unit circle
  std::vector<double> len(bnd.size());
  len[0] = 0.;

  for (int i = 1; i < bnd.size(); i++)
  {
    len[i] = len[i-1] + (V.row(bnd[i-1]) - V.row(bnd[i])).norm();
  }
  double total_len = len[len.size()-1] + (V.row(bnd[0]) - V.row(bnd[bnd.size()-1])).norm();

  UV.resize(bnd.size(),2);
  for (int i = 0; i < bnd.size(); i++)
  {
    double frac = len[i] * 2. * igl::PI / total_len;
    UV.row(map_ij[bnd[i]]) << cos(frac), sin(frac);
  }
'''

' (A libigl function: https://github.com/libigl/libigl/blob/main/include/igl/map_vertices_to_circle.cpp)\n// This file is part of libigl, a simple c++ geometry processing library.\n//\n// Copyright (C) 2014 Stefan Brugger <stefanbrugger@gmail.com>\n//\n// This Source Code Form is subject to the terms of the Mozilla Public License\n// v. 2.0. If a copy of the MPL was not distributed with this file, You can\n// obtain one at http://mozilla.org/MPL/2.0/.\n\n#include "map_vertices_to_circle.h"\n#include "PI.h"\n\nIGL_INLINE void igl::map_vertices_to_circle(\n  const Eigen::MatrixXd& V,\n  const Eigen::VectorXi& bnd,\n  Eigen::MatrixXd& UV)\n{\n  // Get sorted list of boundary vertices\n  std::vector<int> interior,map_ij;\n  map_ij.resize(V.rows());\n\n  std::vector<bool> isOnBnd(V.rows(),false);\n  for (int i = 0; i < bnd.size(); i++)\n  {\n    isOnBnd[bnd[i]] = true;\n    map_ij[bnd[i]] = i;\n  }\n\n  for (int i = 0; i < (int)isOnBnd.size(); i++)\n  {\n    if (!isOnBnd[i])\n    {\n      m

# Modified function: map_vertices_to_square
- Based on the Libigl function above
- Parameterizes the mesh's vertices into a 2D square

In [4]:
def map_vertices_to_square(v, bnd):
    map_ij = np.zeros(v.shape[0], dtype=int)

    isOnBnd = np.full(v.shape[0], False)

    print(isOnBnd.shape, bnd.shape)
    isOnBnd[bnd] = True
    indices = np.arange(bnd.shape[0])
    map_ij[bnd] = indices

    # do the same thing for all interior vertices
    interior = np.where(~isOnBnd)[0] 
    map_ij[interior] = np.arange(interior.shape[0])

    bnd_len = np.zeros(bnd.shape[0], dtype=np.float64)

    for i in range(bnd.shape[0]):
        bnd_len[i] = bnd_len[i-1] + np.linalg.norm(v[bnd[i-1]] - v[bnd[i]])

    total_bnd_len = bnd_len[bnd_len.shape[0]-1] + np.linalg.norm(v[bnd[0]] - v[bnd[bnd.shape[0]-1]])

    bnd_uv = np.zeros((bnd.shape[0], 2))

    frac = 4 * bnd_len / total_bnd_len

    # Taken from https://math.stackexchange.com/a/978493
    def phi(x):
        step0 = np.min(np.vstack((np.ones(x.shape[0]), 3/2 - np.abs(x))), axis=0)
        step1 = np.max(np.vstack((np.zeros(step0.shape[0]), step0)), axis=0)
        return step1

    u_coord = phi(frac - 3/2)
    v_coord = phi(frac - 5/2)

    bnd_uv[map_ij[bnd]] = np.vstack((u_coord, v_coord)).T
    return bnd_uv


In [5]:
p = np.array(igl.cut_to_disk(f))

bnd = igl.boundary_loop(f)

# map boundary to square
bnd_uv = igl.map_vertices_to_circle(v, bnd)
bnd_uv = map_vertices_to_square(v, bnd)

uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
v_p = np.hstack((uv, np.zeros((uv.shape[0],1))))
p2 = mp.plot(v_p, f, shading={"wireframe": "true"})
p2.add_points(np.array([[0.0, 0, 0], [1.0, 0, 0], [0, 1, 0], [1, 1, 0]]), shading={"point_size": 0.1})

plot = mp.plot(v, f)
for cut in p:
    plot.add_lines(v[cut[:-1]], v[cut[1:]], shading={"line_color": "green"})

plot.add_points(v[bnd], shading={"point_size": 0.1})

(11381,) (56,)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

2

In [6]:
# scale uv by image size
# start at 0, 0, go to image_size, image_size
# eval uv(0, 0), and set colour for that pixel

IMG_SIZE = 32

# scale vertex positions to be [0, 255]
scaled_v = v - v.min()
scaled_v *= 255/scaled_v.max()
scaled_v = np.round(scaled_v)

# u: cols, v: rows
scaled_uv = uv * IMG_SIZE-1
colours = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
percentage = 0
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        colours[i,j] = scaled_v[np.argmin(np.linalg.norm(scaled_uv - np.array([j, i]), axis=1))]
    cur_percentage = int(i/IMG_SIZE*100)
    if cur_percentage > percentage:
        print("%d%%" % cur_percentage)
        percentage = cur_percentage

3%
6%
9%
12%
15%
18%
21%
25%
28%
31%
34%
37%
40%
43%
46%
50%
53%
56%
59%
62%
65%
68%
71%
75%
78%
81%
84%
87%
90%
93%
96%


In [7]:
im = Image.fromarray(colours, mode="RGB")

#im.show()
with open("test.png", "wb") as out:
    im.save(out)

In [8]:
raw = np.array(im.getdata())

pts = {}
for xyz in raw:
    xyz = tuple(xyz)
    if xyz not in pts:
        pts[xyz] = len(pts)

recons_faces = []

for i in range(IMG_SIZE-1):
    for j in range(IMG_SIZE-1):
        cur = i*IMG_SIZE+j
        next_row = (i+1)*IMG_SIZE+j
        v2, v3 = [tuple(x) for x in raw[next_row:next_row+2]]
        v0, v1 = [tuple(x) for x in raw[cur:cur+2]]
        recons_faces.append([pts[v0], pts[v1], pts[v2]])
        recons_faces.append([pts[v1], pts[v2], pts[v3]])

recons_faces = np.array(recons_faces)

pts = np.array(list(pts.keys()))

mp.plot(pts, recons_faces)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(135.5, 11…

# Statistics

- Our png file that we generated is 23 KB (for a 256x256 image, pictured above).

- The demo will show a png that is 32x32, which is stored as 1443 B.

- The original .off file storing the camel head is 767 KB (so we achieved significant lossy compression!)